In [3]:
import spark_catalog
import duckdb

In [4]:
catalog = spark_catalog.SparkCatalog(source_name="kexp", lake_location="../../../data/spark/kexp")

:: loading settings :: url = jar:file:/Users/timburns/PycharmProjects/OwlMountain/venv/lib/python3.9/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /Users/timburns/.ivy2/cache
The jars for the packages stored in: /Users/timburns/.ivy2/jars
io.delta#delta-core_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-99f942a1-005e-4f77-b152-b3fff431606a;1.0
	confs: [default]
	found io.delta#delta-core_2.12;2.3.0 in central
	found io.delta#delta-storage;2.3.0 in central
	found org.antlr#antlr4-runtime;4.8 in central
:: resolution report :: resolve 97ms :: artifacts dl 3ms
	:: modules in use:
	io.delta#delta-core_2.12;2.3.0 from central in [default]
	io.delta#delta-storage;2.3.0 from central in [default]
	org.antlr#antlr4-runtime;4.8 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   3   |   0   

23/04/12 22:22:18 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
/Users/timburns/PycharmProjects/OwlMountain/venv/lib/python3.9/site-packages/pyspark/sql/context.py:112: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


In [5]:
kexp_playlist = catalog.get_data_frame("bronze", "KEXP_PLAYLIST")

23/04/12 22:22:25 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


In [6]:
kexp_playlist.show()

+-------------------+--------------------+--------------------+--------------------+--------------------+-------+--------------------+-------+--------+----------+--------------------+--------------------+---------+------------+------------+----------------+----------+---------------+-----+--------------------+--------------------+--------------------+--------+--------------------+-------------+------------------------+-------------------------+
|            airdate|               album|              artist|          artist_ids|             comment|     id|           image_uri|is_live|is_local|is_request|           label_ids|              labels|play_type|recording_id|release_date|release_group_id|release_id|rotation_status| show|            show_uri|                song|       thumbnail_uri|track_id|                 uri|bronze_source|bronze_created_timestamp|bronze_modified_timestamp|
+-------------------+--------------------+--------------------+--------------------+------------------

In [7]:
catalog.sql("select count(*) as play_count from KEXP_PLAYLIST").show()

+----------+
|play_count|
+----------+
|       424|
+----------+



In [8]:
catalog.sql("select id, airdate, artist, song, show from KEXP_PLAYLIST order by id desc").show()

+-------+-------------------+--------------------+--------------------+-----+
|     id|            airdate|              artist|                song| show|
+-------+-------------------+--------------------+--------------------+-----+
|3179544|2023-04-12 19:32:05|       Little Dragon|       Slugs of Love|56683|
|3179543|2023-04-12 19:28:42|The Psychedelic Furs|         Love My Way|56683|
|3179542|2023-04-12 19:24:27|     TV on the Radio|        Shout Me Out|56683|
|3179541|2023-04-12 19:21:11|Algiers feat. Sam...|   I Can't Stand It!|56683|
|3179540|2023-04-12 19:18:42|                null|                null|56683|
|3179539|2023-04-12 19:14:26|El Michels Affair...|       Glorious Game|56683|
|3179538|2023-04-12 19:09:44|    William DeVaughn|Be Thankful for W...|56683|
|3179537|2023-04-12 19:00:30|Khruangbin & Feli...|Time (You and I) ...|56682|
|3179536|2023-04-12 18:56:54|      Herbie Hancock|           Nefertiti|56682|
|3179535|2023-04-12 18:48:52|                null|              

In [9]:
catalog.get_data_frame("bronze", "KEXP_PLAYLIST").printSchema()

root
 |-- airdate: timestamp (nullable = true)
 |-- album: string (nullable = true)
 |-- artist: string (nullable = true)
 |-- artist_ids: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- comment: string (nullable = true)
 |-- id: integer (nullable = true)
 |-- image_uri: string (nullable = true)
 |-- is_live: boolean (nullable = true)
 |-- is_local: boolean (nullable = true)
 |-- is_request: boolean (nullable = true)
 |-- label_ids: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- labels: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- play_type: string (nullable = true)
 |-- recording_id: integer (nullable = true)
 |-- release_date: date (nullable = true)
 |-- release_group_id: integer (nullable = true)
 |-- release_id: integer (nullable = true)
 |-- rotation_status: string (nullable = true)
 |-- show: integer (nullable = true)
 |-- show_uri: string (nullable = true)
 |-- song: string (nullable = tru

In [10]:
catalog.sql("select id, size(artist_ids) from KEXP_PLAYLIST order by size(artist_ids) desc").show()

+-------+----------------+
|     id|size(artist_ids)|
+-------+----------------+
|3179283|               4|
|3179541|               3|
|3179286|               3|
|3179352|               3|
|3179232|               3|
|3179328|               3|
|3179214|               3|
|3179156|               3|
|3179132|               3|
|3179398|               3|
|3179510|               2|
|3179539|               2|
|3179330|               2|
|3179537|               2|
|3179454|               2|
|3179342|               2|
|3179367|               2|
|3179420|               2|
|3179336|               2|
|3179418|               2|
+-------+----------------+
only showing top 20 rows



In [14]:
kexp_playlist = catalog.get_data_frame("bronze", "KEXP_PLAYLIST")

In [15]:
artist = catalog.get_data_frame("silver", "ARTIST")

In [18]:
artist.show(20, False)

+------------------------------------+----------------------------------------------+
|id                                  |artist                                        |
+------------------------------------+----------------------------------------------+
|aa8fc4e5-5332-438a-ba29-51f57a55b084|A Certain Ratio feat. Ellen Beth Abdi         |
|73577e86-4da8-4d76-949d-34c757e053be|A Certain Ratio feat. Ellen Beth Abdi         |
|b7c20887-5042-451d-ac3c-772e25cbe9d8|ABRA                                          |
|bc85da58-52d9-457d-ae8d-5d8d4ec870a9|Aaliyah feat. Timbaland                       |
|daa09819-5da5-4c7a-8bef-eb372bb27ff1|Aaliyah feat. Timbaland                       |
|ba1f396f-eb2c-45bb-9949-973ca060fc86|Abstract Rude + Tribe Unique                  |
|dbd1b11b-0ab2-4afd-8264-c2909237b6fb|Aceyalone                                     |
|f32e763e-be49-44e2-bde0-27c8e5686992|Acid Arab feat. Sofiane Saidi                 |
|8d2a9009-ae32-47fb-b39b-df2861426381|Acid Arab feat. 

In [32]:
catalog.sql('select artist, count(*)'
            ' from KEXP_PLAYLIST'
            ' where artist is not null'
            ' group by artist'
            ' order by count(*) desc').show(20, False)

+--------------------------------+--------+
|artist                          |count(1)|
+--------------------------------+--------+
|Little Dragon                   |4       |
|Screaming Females               |4       |
|Vieux Farka Touré               |4       |
|Herbie Hancock                  |4       |
|Dinner Party feat. Arin Ray     |3       |
|Hannah Jadagu                   |3       |
|Arlo Parks                      |3       |
|De La Soul                      |3       |
|Caroline Polachek               |3       |
|Jungle feat. Erick the Architect|3       |
|U.S. Girls                      |3       |
|Yaeji                           |3       |
|Blondshell                      |3       |
|Dilated Peoples                 |3       |
|M83                             |2       |
|Hole                            |2       |
|OutKast                         |2       |
|The National                    |2       |
|Fort Romeau                     |2       |
|SAULT                          